# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
pd.read_csv("cities.csv")

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed
0,3833367,Ushuaia,AR,-54.8000,-68.3000,49.66,71,75,0.00
1,2015852,Svetlaya,RU,46.5389,138.3297,52.16,69,1,8.99
2,1651810,Airai,TL,-8.9266,125.4092,60.91,91,100,2.80
3,4030556,Rikitea,PF,-23.1203,-134.9692,73.87,66,0,14.79
4,2088659,Panguna,PG,-6.3164,155.4848,68.97,95,64,1.97
...,...,...,...,...,...,...,...,...,...
564,352628,Maţāy,EG,28.4190,30.7792,95.11,15,0,19.48
565,3174050,Province of Mantua,IT,45.1667,10.7833,75.58,34,66,3.00
566,1791779,Wanning,CN,18.8003,110.3967,81.84,89,49,9.89
567,1307741,Myitkyina,MM,25.3833,97.4000,78.76,65,96,5.70


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
